# DATA CLEANING AND PREPARATION FOR MODELING

## MODULE IMPORTS

In [1]:
vars_season= {}

for k in vars.keys():
    lst = vars[k]
    lst = [x.replace('player_match','player_season') if 'player_match' in x else x for x in lst]
    lst = [f"{x}_90" if 'player_season' in x else x for x in lst]
    vars_season[k] = lst


AttributeError: 'builtin_function_or_method' object has no attribute 'keys'

In [2]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import os

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import pickle
from pathlib import Path
from scipy import stats

from stored_vars import vars

import re
import pandas as pd
from difflib import SequenceMatcher

def split_name(name: str):
    '''
    Function takes a string as an argument and splits it based on the amount of words in the string. 
    First word is the first name, other words form the last name.
    '''
    if len(name.split(' ')) == 2:
        first_name = name.split(' ')[0]
        last_name = name.split(' ')[1]
    else:
        first_name = name.split(' ')[0]
        last_name = ' '.join(name.split(' ')[1:])
    
    return first_name, last_name


def clean_name(name):
    """Clean and normalize names for better matching"""
    if pd.isna(name):
        return ""
    # Convert to lowercase, remove accents, extra spaces
    name = str(name).lower().strip()
    name = re.sub(r'[^\w\s-]', '', name)  # Remove special chars except hyphens
    name = re.sub(r'\s+', ' ', name)  # Normalize spaces
    return name

def similarity_score(name1, name2):
    """Calculate similarity between two names"""
    return SequenceMatcher(None, clean_name(name1), clean_name(name2)).ratio()


def map_players_by_name_and_team(tf_df, game_df, name_threshold=0.7, team_boost=0.15):
    """
    Map players using both player_name and team_name for improved accuracy.
    
    Args:
        tf_df (DataFrame): Transfer data with 'player_id', 'player_name', 'team_name'
        game_df (DataFrame): Game data with 'player_id', 'player_name', 'team_name' 
        name_threshold (float): Minimum player name similarity
        team_boost (float): Score boost when team names match well
    
    Returns:
        DataFrame: Mapping with player IDs, names, teams, and match scores
    """
    
    # Get unique players from both datasets
    tf_unique = tf_df[['player_id', 'player_name', 'team_name']].drop_duplicates()
    game_unique = game_df[['player_id', 'player_name', 'team_name']].drop_duplicates()
    
    mappings = []
    
    for _, game_player in game_unique.iterrows():
        game_name = game_player['player_name']
        game_id = game_player['player_id']
        game_team = game_player['team_name']
        
        best_match = None
        best_score = 0
        
        for _, tf_player in tf_unique.iterrows():
            tf_name = tf_player['player_name']
            tf_id = tf_player['player_id']
            tf_team = tf_player['team_name']
            
            # Calculate name similarity
            name_score = similarity_score(game_name, tf_name)
            
            # Skip if name similarity is too low
            if name_score < name_threshold:
                continue
                
            # Calculate team similarity
            team_score = similarity_score(game_team, tf_team)
            
            # Combined score: boost name score if teams match well
            final_score = name_score
            if team_score > 0.8:  # Strong team match
                final_score += team_boost
            
            if final_score > best_score:
                best_score = final_score
                best_match = {
                    'tf_player_id': tf_id,
                    'game_player_id': game_id,
                    'tf_player_name': tf_name,
                    'game_player_name': game_name,
                    'tf_team_name': tf_team,
                    'game_team_name': game_team,
                    'name_similarity': name_score,
                    'team_similarity': team_score,
                    'final_score': final_score,
                    'team_match': team_score > 0.8
                }
        
        if best_match:
            mappings.append(best_match)
    
    return pd.DataFrame(mappings)


def map_players_by_name(tf_players_df, game_data_df, similarity_threshold=0.8):
    """
    Map players between transfer data and game data based on player names only.
    
    Args:
        tf_players_df (DataFrame): Transfer players with 'player_id' and 'player_name'
        game_data_df (DataFrame): Game data with 'player_id' and 'player_name'
        similarity_threshold (float): Minimum similarity score for matching
    
    Returns:
        DataFrame: Mapping with tf_player_id, game_player_id, names, and similarity_score
    """
    
    # Get unique players from both datasets
    tf_unique = tf_players_df[['player_id', 'player_name']].drop_duplicates()
    game_unique = game_data_df[['player_id', 'player_name']].drop_duplicates()
    
    mappings = []
    
    for _, game_player in game_unique.iterrows():
        game_name = game_player['player_name']
        game_id = game_player['player_id']
        
        best_match = None
        best_score = 0
        
        for _, tf_player in tf_unique.iterrows():
            tf_name = tf_player['player_name']
            tf_id = tf_player['player_id']
            
            score = similarity_score(game_name, tf_name)
            
            if score > best_score and score >= similarity_threshold:
                best_score = score
                best_match = {
                    'tf_player_id': tf_id,
                    'game_player_id': game_id,
                    'tf_player_name': tf_name,
                    'game_player_name': game_name,
                    'similarity_score': score
                }
        
        if best_match:
            mappings.append(best_match)
    
    return pd.DataFrame(mappings)



def compute_transfer_values(
    df: pd.DataFrame,
    player_col: str = "player_name",
    value_col: str = "value",
    date_col: str = "marketValue.determined",
    freqs=("half_year", "quarter"),
    drop_na_dates: bool = True,
) -> pd.DataFrame:
    """
    Compute transfer values per player and time interval
    (half-year Jan–Jun/Jul–Dec and/or quarter).

    Parameters
    ----------
    df : pd.DataFrame
        Input data containing player, value, and date columns.
    player_col : str, default "player_name"
        Column name identifying the player.
    value_col : str, default "value"
        Column with transfer value (numeric).
    date_col : str, default "marketValue.determined"
        Column with the date of determination (datetime-like).
    freqs : tuple of {"half_year", "quarter"}
        Which period granularities to include.
    drop_na_dates : bool, default True
        Drop rows where the date cannot be parsed.

    Returns
    -------
    pd.DataFrame
        Columns: [player_col, "year", "period_type", "period", value_col]
    """

    # --- validation ---
    for col in [player_col, value_col, date_col]:
        if col not in df.columns:
            raise KeyError(f"Column '{col}' not found. Available: {list(df.columns)}")

    work = df.copy()
    work[date_col] = pd.to_datetime(work[date_col], errors="coerce")
    if drop_na_dates:
        work = work.dropna(subset=[date_col])

    work[value_col] = pd.to_numeric(work[value_col], errors="coerce").fillna(0)

    work["year"] = work[date_col].dt.year
    work["_month"] = work[date_col].dt.month
    work["half_year"] = work["_month"].map(lambda m: "H1" if m <= 6 else "H2")
    work["quarter"] = work[date_col].dt.quarter.map(lambda q: f"Q{q}")

    outputs = []

    if "half_year" in freqs:
        g = (work
             .groupby([player_col, "year", "half_year"])[value_col]
             .mean()
             .reset_index())
        g = g.rename(columns={"half_year": "period"})
        g["period_type"] = "half_year"
        outputs.append(g[[player_col, "year", "period_type", "period", value_col]])

    if "quarter" in freqs:
        g = (work
             .groupby([player_col, "year", "quarter"])[value_col]
             .mean()
             .reset_index())
        g = g.rename(columns={"quarter": "period"})
        g["period_type"] = "quarter"
        outputs.append(g[[player_col, "year", "period_type", "period", value_col]])

    if not outputs:
        raise ValueError("No valid frequencies selected. Use 'half_year' and/or 'quarter'.")

    out = pd.concat(outputs, ignore_index=True)

    # Sort nicely
    order_map = {"H1": 1, "H2": 2, "Q1": 1, "Q2": 2, "Q3": 3, "Q4": 4}
    out["_sort"] = out["period"].map(order_map).fillna(9)
    out = out.sort_values([player_col, "year", "period_type", "_sort"]).drop(columns="_sort")

    return out


def compute_transfer_values(
    df: pd.DataFrame,
    player_col: str = "player_name",
    value_col: str = "value",
    date_col: str = "marketValue.determined",
    freqs=("half_year", "quarter"),
    drop_na_dates: bool = True,
) -> pd.DataFrame:
    """
    Compute transfer values per player and time interval
    (half-year Jan–Jun/Jul–Dec and/or quarter), including mean age.

    Parameters
    ----------
    df : pd.DataFrame
        Input data containing player, value, age, and date columns.
    player_col : str, default "player_name"
        Column name identifying the player.
    value_col : str, default "value"
        Column with transfer value (numeric).
    date_col : str, default "marketValue.determined"
        Column with the date of determination (datetime-like).
    freqs : tuple of {"half_year", "quarter"}
        Which period granularities to include.
    drop_na_dates : bool, default True
        Drop rows where the date cannot be parsed.

    Returns
    -------
    pd.DataFrame
        Columns: [player_col, "year", "period_type", "period", value_col, "age"]
    """

    # --- validation ---
    for col in [player_col, value_col, date_col, "age"]:
        if col not in df.columns:
            raise KeyError(f"Column '{col}' not found. Available: {list(df.columns)}")

    work = df.copy()
    work[date_col] = pd.to_datetime(work[date_col], errors="coerce")
    if drop_na_dates:
        work = work.dropna(subset=[date_col])

    work[value_col] = pd.to_numeric(work[value_col], errors="coerce").fillna(0)
    work["age"] = pd.to_numeric(work["age"], errors="coerce")

    work["year"] = work[date_col].dt.year
    work["_month"] = work[date_col].dt.month
    work["half_year"] = work["_month"].map(lambda m: "H1" if m <= 6 else "H2")
    work["quarter"] = work[date_col].dt.quarter.map(lambda q: f"Q{q}")

    outputs = []

    if "half_year" in freqs:
        g = (
            work
            .groupby([player_col, "year", "half_year"], as_index=False)
            .agg({value_col: "mean", "age": "mean"})
            .rename(columns={"half_year": "period"})
        )
        g["period_type"] = "half_year"
        outputs.append(g[[player_col, "year", "period_type", "period", value_col, "age"]])

    if "quarter" in freqs:
        g = (
            work
            .groupby([player_col, "year", "quarter"], as_index=False)
            .agg({value_col: "mean", "age": "mean"})
            .rename(columns={"quarter": "period"})
        )
        g["period_type"] = "quarter"
        outputs.append(g[[player_col, "year", "period_type", "period", value_col, "age"]])

    if not outputs:
        raise ValueError("No valid frequencies selected. Use 'half_year' and/or 'quarter'.")

    out = pd.concat(outputs, ignore_index=True)

    # Sort nicely
    order_map = {"H1": 1, "H2": 2, "Q1": 1, "Q2": 2, "Q3": 3, "Q4": 4}
    out["_sort"] = out["period"].map(order_map).fillna(9)
    out = out.sort_values([player_col, "year", "period_type", "_sort"]).drop(columns="_sort")

    return out



def add_period_columns(
    df: pd.DataFrame,
    date_col: str,
    out_year: str = "year",
    out_quarter: str = "quarter",
    out_half: str = "half_year",
    drop_na_dates: bool = False,
    inplace: bool = False,
) -> pd.DataFrame:
    """
    Add year, quarter (Q1..Q4), and half-year (H1/H2) columns to a DataFrame that has a date column.
    
    Parameters
    ----------
    df : pd.DataFrame
        Input dataframe.
    date_col : str
        Name of the date column in df (datetime-like or string parseable to datetime).
    out_year : str, default "year"
        Output column name for year.
    out_quarter : str, default "quarter"
        Output column name for quarter label ("Q1".."Q4").
    out_half : str, default "half_year"
        Output column name for half-year label ("H1"/"H2").
    drop_na_dates : bool, default False
        If True, drop rows where date parsing failed; otherwise, keep rows with NaT.
    inplace : bool, default False
        If True, modify df in place and return it. If False, work on a copy.
        
    Returns
    -------
    pd.DataFrame
        DataFrame with added period columns.
    """
    work = df if inplace else df.copy()
    if date_col not in work.columns:
        raise KeyError(f"Date column '{date_col}' not found. Available: {list(work.columns)}")
    
    work[date_col] = pd.to_datetime(work[date_col], errors="coerce")
    if drop_na_dates:
        work = work.dropna(subset=[date_col])
    
    work[out_year] = work[date_col].dt.year
    work[out_quarter] = work[date_col].dt.quarter.map(lambda q: f"Q{q}" if pd.notna(q) else None)
    work["_month__tmp"] = work[date_col].dt.month
    work[out_half] = work["_month__tmp"].map(lambda m: "H1" if pd.notna(m) and m <= 6 else ("H2" if pd.notna(m) else None))
    work = work.drop(columns=["_month__tmp"])
    
    return work



## DATA LOAD & PREPARATION

In [7]:
## load season stats and keep relevant competitions:
seasonStats = pd.read_csv('datasets/gamedata/SeasonStats.csv',sep=';')

keep_comps = ['Bundesliga','Eliteserien','Eredivisie','Jupiler Pro League','Super League','Primeira Liga']
seasonStats = seasonStats[seasonStats['competition_name'].isin(keep_comps)]

### Filter on minutes played for the season and calculate number of 90s played:
seasonStats = seasonStats[seasonStats['player_season_minutes'] >= 600]
seasonStats['#_90s'] = (seasonStats['player_season_minutes'] / 90).round(0).astype(int)

## Clean up the positions
seasonStats['primary_position_group'] = (
    seasonStats['primary_position']
    .str.replace('Right','')
    .str.replace('Left','')
    .str.replace('Centre Defensive','Defensive')
    .str.replace('Centre Attacking','Attacking')
    .str.replace('Centre Midfielder','Midfielder')
    .str.lstrip(' ')
    .str.rstrip(' ')
)

seasonStats['secondary_position_group'] = (
    seasonStats['secondary_position']
    .str.replace('Right','')
    .str.replace('Left','')
    .str.replace('Centre Defensive','Defensive')
    .str.replace('Centre Attacking','Attacking')
    .str.replace('Centre Midfielder','Midfielder')
    .str.lstrip(' ')
    .str.rstrip(' ')
)
seasonStats = seasonStats.dropna(how='all',axis=1)
seasonStats = seasonStats.dropna(subset='primary_position')
seasonStats = seasonStats[seasonStats.primary_position != 'Goalkeeper']
seasonStats = seasonStats.drop(
    columns=[
        'player_season_obv_gk_90','player_season_ot_shots_faced_ratio',
        'player_season_penalty_conversion_ratio','player_known_name',
        'player_first_name','player_last_name','player_weight','secondary_position_group',
        'player_season_gsaa_ratio','secondary_position','player_height','primary_position',
        'account_id','team_id','player_season_gsaa_90','player_season_360_minutes','player_season_most_recent_match'
        ]
)

vars_with_na = seasonStats.isna().sum().sort_values(ascending=False).index.tolist()
print(len(vars_with_na))
id_vars = ['player_season_minutes','player_id','country_id','birth_date',
 'player_female',
 'season_name',
 'season_id',
 'competition_name',
 'competition_id','player_season_clcaa',
 'team_name', 'player_name', 'primary_position_group',
 'team_name', 'player_name', 'primary_position_group']
print(len(id_vars))

vars_with_na = [x for x in vars_with_na if x not in id_vars]
print(len(vars_with_na))


for var in vars_with_na:
    seasonStats[var] = seasonStats[var].fillna(seasonStats[var].mean())



156
16
143


/var/folders/gr/grh3033930d1vhxcd4gdjy6m0000gn/T/ipykernel_16621/2352698896.py:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  seasonStats = pd.read_csv('datasets/gamedata/SeasonStats.csv',sep=';')


In [ ]:
tfTeams['competition_id'].unique().tolist()


['BE1',
 'competition_id',
 'SE1',
 'A1',
 'DK1',
 'JAP1',
 'NL1',
 'NO1',
 'PO1',
 'C1',
 'MLS1']

In [20]:
## load tf team data
tfTeams = pd.read_csv('datasets/transfermarkt_teams.csv',sep=',')
tfTeams['team_name'] = tfTeams['team_url'].str.split('/').str[3].str.replace('-',' ')
tfTeams['team_id'] = tfTeams['team_url'].str.split('/').str[-3]

keep_comps_ids = ['A1','DK1','NL1','BE1','C1','PO1']
tfTeams = tfTeams[tfTeams['competition_id'].isin(keep_comps_ids)]

## load tf player links
links = pd.read_csv('datasets/transfermarkt_player_links.csv',sep=',')
links['player_name'] = links['player_url'].str.split('/').str[1].str.replace('-',' ')

## loaf tf MV data
tfValues1 = pd.read_csv('datasets/transfermarkt_players_values.csv',sep=',')
print(len(tfValues1))
tfValues2 = pd.read_csv('datasets/transfermarkt_players_values_2.csv',sep=',')
print(len(tfValues2))

tfValues = pd.concat([tfValues1,tfValues2],ignore_index=True)
print(len(tfValues))

tfValues = tfValues.drop_duplicates(subset=['playerId','marketValue.determined'])
print(len(tfValues))

tfValues['clubId'] = tfValues['clubId'].astype(str)
tfValues = tfValues[~tfValues['marketValue.compact.suffix'].isna()]
tfValues = tfValues.merge(links[['player_id','player_name']],
                          left_on='playerId',
                          right_on='player_id',
                          how='left')
tfValues = tfValues.merge(tfTeams[['team_id','team_name']],left_on = 'clubId',
                          right_on='team_id',how='left')

tfValues = tfValues[tfValues['marketValue.compact.content'] != 'marketValue.compact.content']


12978
123243
136221
114171


In [22]:
seasons = pd.read_csv('datasets/transfermarkt_seasons_2.csv',sep=',')
seasons = seasons.competition_id.unique().tolist()

In [23]:
## keep needed cols and set values to float
tfValues = tfValues[['player_name','player_id','age','team_name','marketValue.value','marketValue.compact.content',
                     'marketValue.compact.suffix','marketValue.determined']]

tfValues['marketValue.compact.content'] = tfValues['marketValue.compact.content'].str.replace(',','.')
tfValues['marketValue.compact.content'] = tfValues['marketValue.compact.content'].astype(float)

## use suffix col to create a correct value in format "x.y" in millions.
valueList = []
for i in range(len(tfValues)):
    val = tfValues['marketValue.compact.content'].iloc[i]
    suff = tfValues['marketValue.compact.suffix'].iloc[i]

    if suff == 'K':
        valueList.append(val/1000)
    elif suff == 'M':
        valueList.append(val)

tfValues['value'] = valueList

/var/folders/gr/grh3033930d1vhxcd4gdjy6m0000gn/T/ipykernel_16621/3779671005.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tfValues['marketValue.compact.content'] = tfValues['marketValue.compact.content'].str.replace(',','.')
/var/folders/gr/grh3033930d1vhxcd4gdjy6m0000gn/T/ipykernel_16621/3779671005.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tfValues['marketValue.compact.content'] = tfValues['marketValue.compact.content'].astype(float)
/var/folders/gr/grh3033930d1vhxcd4gdjy6m0000gn/T/ipykerne

## Mapping datasources

In [24]:
## use mapping function to find player names that match. After some visual inspection 0.9 seems like a good threshold.
mapped_players = map_players_by_name_and_team(seasonStats, tfValues, name_threshold=0.7, team_boost=0.15)
mapped_players_solid = mapped_players[mapped_players.final_score >= 0.9]

In [27]:
mapped_players_solid

,tf_player_id,game_player_id,tf_player_name,game_player_name,tf_team_name,game_team_name,name_similarity,team_similarity,final_score,team_match
0,34930,428920.0,Kevin Mac Allister,kevin mac allister,Union Saint-Gilloise,NaN,1.0,0.000000,1.00,False
1,34930,428920.0,Kevin Mac Allister,kevin mac allister,Union Saint-Gilloise,royale union saint gilloise,1.0,0.808511,1.15,True
2,52221,660746.0,Fedde Leysen,fedde leysen,PSV Eindhoven II,NaN,1.0,0.000000,1.00,False
3,52221,660746.0,Fedde Leysen,fedde leysen,Union Saint-Gilloise,royale union saint gilloise,1.0,0.808511,1.15,True
4,3880,244221.0,Christian Burgess,christian burgess,Union Saint-Gilloise,NaN,1.0,0.000000,1.00,False
...,...,...,...,...,...,...,...,...,...,...
1807,32977,535551.0,Bartosz Bialek,bartosz bialek,Vitesse,kas eupen,1.0,0.250000,1.00,False
1808,5514,90024.0,Alfred Finnbogason,alfred finnbogason,AS Eupen,NaN,1.0,0.000000,1.00,False
1809,5514,90024.0,Alfred Finnbogason,alfred finnbogason,AS Eupen,sc heerenveen,1.0,0.476190,1.00,False
1810,5514,90024.0,Alfred Finnbogason,alfred finnbogason,AS Eupen,lyngby bk,1.0,0.117647,1.00,False


In [31]:
## map id from games to tf values table
mapped_players_solid2 = (
    mapped_players_solid
    .drop_duplicates(subset=['game_player_id'],keep='first')
    [['tf_player_id','game_player_id','game_player_name']]
    .rename(columns={'tf_player_id':'sb_id'})
    )

print(len(mapped_players_solid2))
tfValues2 = (
    tfValues
    .merge(mapped_players_solid2,left_on='player_id',right_on='game_player_id',how='left')
)
tfValues2 = tfValues2.dropna(subset='game_player_id')
tfValues2 = tfValues2.drop(columns=['game_player_id','game_player_name'])

445


In [32]:
def add_season_and_avg_age(
    df: pd.DataFrame,
    *,
    date_col: str,                 # e.g. 'marketvalue.Determined'
    player_col: str,               # e.g. 'player_id' or 'player_name'
    age_col: str = "age",          # existing rounded age column
    season_col: str = "season",
    avg_age_col: str = "avg_age_season",
    date_format: str | None = "%Y-%m-%d",  # None = auto-infer
    round_age: int | None = 2              # decimals for the average; None = no rounding
) -> pd.DataFrame:
    """
    Adds a 'season' column based on `date_col` (Jul 1 -> Jun 30 => 'YYYY/YYYY+1'),
    and an average age per (player, season) computed from the existing `age_col`.
    """

    # ---- Parse record dates robustly ----
    raw = df[date_col].astype(str).str.strip()
    raw = raw.replace({"": np.nan, "nan": np.nan, "NaN": np.nan, "None": np.nan})
    if date_format is not None:
        dates = pd.to_datetime(raw, format=date_format, errors="coerce")
    else:
        dates = pd.to_datetime(raw, errors="coerce", infer_datetime_format=True)
    if dates.isna().mean() > 0.25:
        fallback = pd.to_datetime(raw, errors="coerce", infer_datetime_format=True)
        if fallback.notna().sum() > dates.notna().sum():
            dates = fallback

    # ---- Season: YYYY/YYYY+1 ----
    year = dates.dt.year
    month = dates.dt.month
    start_year = year.where(month >= 7, year - 1)
    end_year = start_year + 1
    season = (start_year.astype("Int64").astype(str) + "/" + end_year.astype("Int64").astype(str))
    season = season.where(dates.notna(), pd.NA)
    df[season_col] = season

    # ---- Average age per player-season (from existing age column) ----
    # Coerce age to numeric; ignore non-numeric as NaN
    age_numeric = pd.to_numeric(df[age_col], errors="coerce")
    # Compute per (player, season) mean and broadcast back with transform
    avg_age = df.assign(_age=age_numeric).groupby([player_col, season_col], dropna=False)["_age"].transform("mean")
    if round_age is not None:
        avg_age = avg_age.round(round_age)
    df[avg_age_col] = avg_age

    return df


def add_season(
    df: pd.DataFrame,
    *,
    date_col: str,                 # e.g. 'marketvalue.Determined'
    season_col: str = "season",
    date_format: str | None = "%Y-%m-%d",  # None = auto-infer
) -> pd.DataFrame:
    """
    Adds a 'season' column based on `date_col` (Jul 1 -> Jun 30 => 'YYYY/YYYY+1').
    """

    # ---- Parse record dates robustly ----
    raw = df[date_col].astype(str).str.strip()
    raw = raw.replace({"": np.nan, "nan": np.nan, "NaN": np.nan, "None": np.nan})
    if date_format is not None:
        dates = pd.to_datetime(raw, format=date_format, errors="coerce")
    else:
        dates = pd.to_datetime(raw, errors="coerce", infer_datetime_format=True)

    if dates.isna().mean() > 0.25:
        # fallback attempt with flexible parsing
        fallback = pd.to_datetime(raw, errors="coerce", infer_datetime_format=True)
        if fallback.notna().sum() > dates.notna().sum():
            dates = fallback

    # ---- Season: YYYY/YYYY+1 ----
    year = dates.dt.year
    month = dates.dt.month
    start_year = year.where(month >= 7, year - 1)
    end_year = start_year + 1
    season = (start_year.astype("Int64").astype(str) + "/" +
              end_year.astype("Int64").astype(str))
    season = season.where(dates.notna(), pd.NA)

    df[season_col] = season
    return df


In [34]:
tfValues3 = add_season_and_avg_age(tfValues2,date_col='marketValue.determined',player_col='player_id',age_col='age')

In [35]:
tfValues3 = (
    tfValues3[['player_name','sb_id','season','value','marketValue.determined','age']]
    .sort_values(by=['player_name','sb_id','marketValue.determined'],ascending=True)

)

tfValues4 = tfValues3.drop_duplicates(subset=['player_name','season'],keep='last')
tfValues4 = tfValues4.drop(columns=['player_name'])
tfValues4

,sb_id,season,value,marketValue.determined,age
3205,134805.0,2019/2020,0.325,2020-04-08,19
3207,134805.0,2020/2021,1.500,2021-02-17,20
3212,134805.0,2021/2022,2.000,2022-04-12,21
3216,134805.0,2022/2023,3.300,2023-06-09,22
3220,134805.0,2023/2024,2.500,2024-06-04,23
...,...,...,...,...,...
22154,16341.0,2020/2021,12.000,2021-06-07,21
22156,16341.0,2021/2022,8.000,2022-06-07,22
22166,16341.0,2022/2023,4.500,2023-06-23,23
22176,16341.0,2023/2024,5.000,2024-06-04,24


In [36]:
seasonStats_merged = seasonStats.merge(tfValues4,left_on=['player_id','season_name'],right_on=['sb_id','season'],how='left')
print(len(seasonStats_merged))

seasonStats_merged2 = seasonStats_merged.dropna(subset=['value'])
print(len(seasonStats_merged2))

6958
966


In [37]:
## add club elo as a factor for that period:
dfELO = pd.read_csv('datasets/Elo_teams_mapped.csv',sep=',')
dfELO2 = add_season(dfELO,date_col='Date_ref')

/var/folders/gr/grh3033930d1vhxcd4gdjy6m0000gn/T/ipykernel_16621/2101697413.py:71: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  fallback = pd.to_datetime(raw, errors="coerce", infer_datetime_format=True)


In [42]:
dfElo3 = dfELO2[['Match1','season','Elo']].groupby(['Match1','season']).mean().reset_index().rename(columns={'Match1':'team_name'})

seasonStats_merged3 = seasonStats_merged2.merge(dfElo3,on=['team_name','season'],how='left')


In [43]:
def calculate_player_improvement_with_flag(data: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate percent change for all 'player_match_' columns compared to the previous period for each player.
    Adds a column 'is_baseline' = 1 if it's the first period for a player, else 0.
    """
    # Find all columns that start with 'player_match_'
    season_cols = [col for col in data.columns if col.startswith("player_season_")]
    season_cols = [col for col in season_cols if 'minutes' not in col]
    
    # Sort by player and period
    data_sorted = data.sort_values(by=["player_id", "season"]).copy()
    
    # Group by player and calculate percent change vs. previous period
    for col in season_cols:
        data_sorted[f"{col}_improvement"] = data_sorted.groupby("player_id")[col].pct_change() * 100
    
    # Mark baseline periods (first for each player)
    data_sorted["is_baseline"] = data_sorted.groupby("player_id")["season"].rank(method="first").eq(1).astype(int)
    
    return data_sorted

## Apply improvement function
seasonStats_merged4 = calculate_player_improvement_with_flag(seasonStats_merged3)

/var/folders/gr/grh3033930d1vhxcd4gdjy6m0000gn/T/ipykernel_16621/3333517254.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_sorted[f"{col}_improvement"] = data_sorted.groupby("player_id")[col].pct_change() * 100
/var/folders/gr/grh3033930d1vhxcd4gdjy6m0000gn/T/ipykernel_16621/3333517254.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_sorted[f"{col}_improvement"] = data_sorted.groupby("player_id")[col].pct_change() * 100
/var/folders/gr/grh3033930d1vhxcd4gdjy6m0000gn/T/ipykernel_16621/3333517254.py:15: Perform

In [44]:
seasonStats_merged4['team_name'].unique()

array(['Benfica', 'RSC Anderlecht', 'Standard Liège', 'Antwerp',
       'Almere City', 'Beerschot', 'Sporting Charleroi', 'Gent',
       'Union Saint-Gilloise', 'Sint-Truiden', 'Mechelen', 'AS Eupen',
       'BSC Young Boys', 'OH Leuven', 'Westerlo', 'Club Brugge',
       'KRC Genk', 'Kortrijk', 'RWDM', 'Servette', 'AZ Alkmaar', 'Ajax',
       'Cercle Brugge', 'Zulte Waregem', 'Feyenoord', 'KV Oostende',
       'FC Lausanne Sport', 'Beveren', 'Sturm Graz', 'Go Ahead Eagles',
       'FC Groningen', 'Fortuna Sittard', 'Dender', 'PEC Zwolle',
       'Willem II', 'RKC Waalwijk', 'RFC Seraing', 'Rio Ave',
       'Heracles Almelo', 'FC Utrecht', 'Twente', 'FC Emmen',
       'FC Volendam', 'Vitesse', 'Sparta Rotterdam', 'SC Heerenveen',
       'Santa Clara', 'Grasshopper', 'Luzern', 'Royal Excel Mouscron',
       'NAC Breda', 'WSG Tirol', 'Basel', 'Austria Wien', 'NEC',
       'Sporting CP', 'FC Arouca', 'LASK', 'Lugano',
       'FC Stade Lausanne-Ouchy', 'Rapid Wien', 'Sion'], dtype=object)

In [45]:
seasonStats_merged4.to_csv('model_data/merged_season_data.csv',sep=',',index=False)